In [22]:
!pip install requests #requestsをインストール

In [23]:
!pip install beautifulsoup4 #beautifulsoup4をインストール

In [2]:
import os , os.path #osを読み込む
import sys
import pandas as pd #xlsxのファイルをcsvにする
import ast
import requests
from bs4 import BeautifulSoup
import json
from urllib import request
import csv

#ここから係り受け構文解析
APPID = "dj00aiZpPUVzanI5WmtzbjExVyZzPWNvbnN1bWVyc2VjcmV0Jng9NTQ-"  # <-- ここにあなたのアプリケーションIDを設定してください。
URL = "https://jlp.yahooapis.jp/DAService/V2/parse"

#post(???)で係り受け解析をした結果を得ることができる
def post(query):
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Yahoo AppID: {}".format(APPID),
    }
    param_dic = {
        "id": "1234-1",
        "jsonrpc": "2.0",
        "method": "jlp.daservice.parse",
        "params": {"q": query},
    }
    params = json.dumps(param_dic).encode()
    req = request.Request(URL, params, headers)
    with request.urlopen(req) as res:
        body = res.read()
    return body.decode()

In [25]:
#辞書型 {鍵：値}
cameras = {"EOS R5_body":["K0001272146",3] , "EOS Kiss M_doublekit":["J0000027006",3] , "EOS R_body":["K0001086545",11] , "EOS RP_body":["K0001129952",3] , "EOS M5_body":["K0000910398",4] , "EOS Kiss M_body":["J0000027002",3] , "EOS Kiss M_STMkit":["J0000001579",2] , "EOS M_doublekit":["J0000001581",15] , "EOS M2_doublekit":["J0000011374",4] , "EOS M3_body":["J0000014693",3]}

In [ ]:
#dic = {'key1': 110, 'key2': 270, 'key3': 350}　　
#for key in dic:　　 
    #print(key) # key1〜key3が出力される　　 
    #print(dic[key]) # それぞれのkeyに対応する値が出力される

In [ ]:
#d = {"D5500" : ["123456",5]}
#print(d["D5500"][1])

In [26]:
for camera in cameras:
    for page in range(1,cameras[camera][1]+1): #range(1,4)は1,2,3なので＋1をする
        r = requests.get("https://review.kakaku.com/review/{0}/page={1}/#tab".format(cameras[camera][0],str(page))) #requestを使ってurlを呼び出す parserでrを分析  {0}{1}はformatの何番目にその要素があるか
        soup = BeautifulSoup(r.content, "html.parser") #BeautifulSoupのアルゴリズムを使う
        wordclass = soup.find_all("p","revEntryCont") #pでくくられたrevEntryCont（レビュー本文）を全部探す
        for person,item in enumerate(wordclass,1): #for i, name in enumerate(l, 1):  print(i, name)  1 Alice  2 Bob  3 Charlie　　
            word = str(item.text).replace('\\u3000', ' ') #strにすることで1などの数字は全部文字列型にする
            #print(person)
            #print(word)
            #response = post(word)
            response = ast.literal_eval(post(word))
            #print(response)
            with open("./review/{0}_{1}_{2}.csv".format(camera,str(page),person), "w", newline="")as fp: #newline=""はcsvファイルの一行詰めるため
                for element in response["result"]["chunks"]:
                    #print(element["tokens"])
                    #lst = list(csv.reader(fp))
                    writer = csv.writer(fp)
                    writer.writerows(element["tokens"])
                #file = open("./review/{0}_{1}_{2}.txt".format(camera,str(page),person), "w", encoding = 'cp932', errors = 'ignore')
                #file.write(word)
                #file.close()
            with open("./review/{0}_{1}_{2}.csv".format(camera,str(page),person), encoding = 'cp932', errors = 'ignore') as fp:
                lst = []
                reader = csv.reader(fp)
                for row in reader:
                    lst.append(row[2])#C列
            #print(lst)#listはレビューを係り受け解析し、csvにし、Ⅽ列（原形）を抜き出したもの

In [29]:
#print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
#print(" ------------------------------------ ")
#for var_name in dir():
    #if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 100: #ここだけアレンジ
        #print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [30]:
#del BeautifulSoup, APPID, cameras, post, word, wordclass, lst, element, response #容量大きいものを削除

In [16]:
df1 = pd.read_excel("new評価ワード.xlsx", sheet_name=0)
df1.to_csv("new評価ワード.csv", index=False, header=None, encoding='utf_8_sig')
with open("new評価ワード.csv", encoding='utf_8_sig') as fp:
    hyokaword_lst = []
    reader = csv.reader(fp)
    for row in reader:
        hyokaword_lst.append(row[0])#A列
mojiretsu = ''.join(hyokaword_lst)
#print(hyokaword_lst)
#print(type(hyokaword_lst))
#print(mojiretsu)
response = ast.literal_eval(post(mojiretsu))
#print(response)
with open("new評価ワード(係り受け済).csv", "w", newline="")as fp:# newline=""はcsvファイルの一行詰めるため
    writer = csv.writer(fp)
    hyokaword_kakariuke_lst = []
    for element in response["result"]["chunks"]:
        hyokaword_kakariuke_lst.append(element["tokens"])
        hyokaword_kakariuke_lst.append(element["id"])
        hyokaword_kakariuke_lst.append(element["head"])
    print(hyokaword_kakariuke_lst)
        #print(element["tokens"])
        #lst = list(csv.reader(fp))
        #writer.writerows(element["tokens"])
        #writer.writerow(element["head"])
        #writer.writerow(element["id"])
    writer.writerows(hyokaword_kakariuke_lst)
        
#with open("new評価ワード(係り受け済).csv") as fp:
#    hyokaword_genkei_lst = []
#    reader = csv.reader(fp)
#    for row in reader:
#        hyokaword_genkei_lst.append(row[2])#C列
    #print(hyokaword_genkei_lst)

#    word_dict = {}
#    for hyokaword_genkei in hyokaword_genkei_lst:
#        word_dict[hyokaword_genkei] = 0
#    print(word_dict)

[[['キヤノン', 'きやのん', 'キヤノン', '名詞', '組織名', '*', '*'], ['らしい', 'らしい', 'らしい', '助動詞', '*', 'イ形容詞イ段', '基本形']], 0, 1, [['キヤノン', 'きやのん', 'キヤノン', '名詞', '組織名', '*', '*'], ['らしく', 'らしく', 'らしい', '助動詞', '*', 'イ形容詞イ段', '基本連用形'], ['ない', 'ない', 'ない', '接尾辞', '形容詞性述語接尾辞', 'イ形容詞アウオ段', '基本形']], 1, 7, [['ニコン', 'にこん', 'ニコン', '名詞', '組織名', '*', '*'], ['らしい', 'らしい', 'らしい', '助動詞', '*', 'イ形容詞イ段', '基本形']], 2, 7, [['フジ', 'ふじ', 'フジ', '名詞', '普通名詞', '*', '*'], ['らしい', 'らしい', 'らしい', '助動詞', '*', 'イ形容詞イ段', '基本形']], 3, 7, [['ソニー', 'そにー', 'ソニー', '名詞', '組織名', '*', '*'], ['らしい', 'らしい', 'らしい', '助動詞', '*', 'イ形容詞イ段', '基本形']], 4, 7, [['パナソニック', 'ぱなそにっく', 'パナソニック', '名詞', '組織名', '*', '*'], ['らしい', 'らしい', 'らしい', '助動詞', '*', 'イ形容詞イ段', '基本形']], 5, 7, [['LUMIX', 'LUMIX', 'LUMIX', '未定義語', 'アルファベット', '*', '*'], ['らしい', 'らしい', 'らしい', '助動詞', '*', 'イ形容詞イ段', '基本形']], 6, 7, [['手', 'て', '手', '名詞', '普通名詞', '*', '*'], ['に', 'に', 'に', '助詞', '格助詞', '*', '*']], 7, 9, [['なじむ', 'なじむ', 'なじむ', '動詞', '*', '子音動詞マ行', '基本形'], ['な', 'な', 'な', '名詞', '普通名詞', 

Error: iterable expected, not int

In [28]:
df2 = pd.read_excel("new評価表現辞書.xlsx", sheet_name=0)
df2.to_csv("new評価表現辞書.csv", index=False, header=None, encoding='utf_8_sig')
with open("new評価表現辞書.csv", encoding='utf_8_sig') as fp:
    hyokajisyo_lst = []
    reader = csv.reader(fp)
    for row in reader:
        hyokajisyo_lst.append(row[0])#A列
mojiretsu2 = ''.join(hyokajisyo_lst)
#print(hyokajisyo_lst)
response = ast.literal_eval(post(mojiretsu2))
#print(response)
with open("new評価表現辞書(係り受け済).csv", "w", newline="")as fp:# newline=""はcsvファイルの一行詰めるため
    writer = csv.writer(fp)
    for element in response["result"]["chunks"]:
        #print(element["tokens"])
        #lst = list(csv.reader(fp))
        writer.writerows(element["tokens"])

In [17]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 100: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|                    APPID|       105|
|            BeautifulSoup|      1056|
|                       In|       264|
|                      Out|       240|
|                      df1|    120458|
|                  element|       240|
|                       fp|       216|
|  hyokaword_kakariuke_lst|      3304|
|            hyokaword_lst|      1928|
|                mojiretsu|      1952|
|                     post|       136|
|                   reader|       104|
|                 response|       240|
|                      row|       192|


In [18]:
del hyokaword_kakariuke_lst, hyokaword_lst #容量大きいものを削除

In [34]:
kanseis = ["かっこいい", "大きい", "重い", "使いやすい", "グリップ", "安定", "携帯性が良い", "操作性が悪い", "グリップが良い", "ピントが合う", ]
mydict = {}
for kansei in kanseis:
    mydict[kansei] = 0
print(mydict)

{'かっこいい': 0, '大きい': 0, '重い': 0, '使いやすい': 0, 'グリップ': 0, '安定': 0, '携帯性が良い': 0, '操作性が悪い': 0, 'グリップが良い': 0, 'ピントが合う': 0}


In [ ]:
#lst = ["suzuki" , "tanaka" , "satou" , "maeda", "tanaka" , "satou" , "maeda"]

In [ ]:
#lst.count("maeda")